In [2]:
import heapq, random

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

In [3]:
class Person:
    def __init__(self, height: int) -> None:
        """
        height: height in inches rounded to nearest whole number
        """
        self.height = height
        self.wingspan = height * 1.06
        self.reach = height * 1.35
        self.leg_length = height * 0.5

In [4]:
class Hold:
    def __init__(self, x: int, y: int, diff: float, width: float, height: float, angle: int):
        #Coords = top left corner
        self.x = x
        self.y = y
        self.diff = diff
        self.width = width
        self.height = height
        self.angle = angle
    def getCenter(self):
        return (self.coords[0] + self.width/2, self.coords[1] + self.height/2)
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __gt__(self, other):
        return self.y > other.y
    def __lt__(self, other):
        return self.y < other.y
    def __ge__(self, other):
        return self.y >= other.y
    def __le__(self, other):
        return self.y <= other.y
    
    def __repr__(self):
        return (f"""Hold: Top left at {self.x}, {self.y}
                Width = {self.width}, Height = {self.height}
                Difficulty = {round(self.diff, 2)}/10, Angle = {self.angle} degrees\n""")

In [5]:
from enum import Enum

class LimbName(Enum):
    LEFT_HAND = "Left Hand"
    RIGHT_HAND = "Right Hand"
    LEFT_LEG = "Left Leg"
    RIGHT_LEG = "Right Leg"


In [6]:
class Limb:
    def __init__(self, name: LimbName, strength: int, flexibility: int, hold: Hold):
        self.name, self.strength, self.flexibility, self.hold = name, strength, flexibility, hold
    def __repr__(self):
        return (f"{self.name} at {self.hold}")

In [7]:
class Route:
    def __init__(self, holds: Hold, start1: Hold, start2: Hold, finish: Hold):
        self.holds = holds
        self.start_hold1 = start1
        self.start_hold2 = start2
        self.finish_hold = finish

In [138]:
import math
from copy import copy

class State:
    def __init__(self, lf: Limb, rf: Limb, lh: Limb, rh: Limb, person: Person, route: Route):
        self.lf, self.rf, self.lh, self.rh = lf, rf, lh, rh
        self.person = person
        self.moves = []
        self.costs = []
        self.route = route
        self.wall_height_inches = 180 # Represents real height of wall <- need to measure
        self.wall_height_pixels = 3321 #placeholder <- Need to extract from image

    # Not sure where to put this
    def inches_to_pixels(self, inches: int):
        return (self.wall_height_pixels / self.wall_height_inches) * inches

    def __eq__(self, other):
        return ((self.lf.hold == other.lf.hold and self.rf.hold == other.rf.hold 
                and self.rh.hold == other.rh.hold and self.lh.hold == other.lh.hold)
                or (self.lf.hold == other.rf.hold and self.rf.hold == other.lf.hold 
                and self.rh.hold == other.lh.hold and self.lh.hold == other.rh.hold))

    def __repr__(self):
        return f"{self.moves}"

    def getStateSuccessors(self):
        succs = []
        limbs = [self.lf, self.rf, self.lh, self.rh]
        for i in range(len(limbs)):
            neighs = self.getNeighbors(limbs[i])
            for neigh in neighs:
                #print("Hi neighbor")
                new_state = copy(self)
                if i == 0:
                    new_state.lf = Limb(LimbName.LEFT_LEG, 2.5, 8, neigh)
                    action = (self.lf, neigh)
                if i == 1:
                    new_state.rf = Limb(LimbName.RIGHT_LEG, 2.5, 8, neigh)
                    action = (self.rf, neigh)
                if i == 2:
                    new_state.lh = Limb(LimbName.LEFT_HAND, 8, 2, neigh)
                    action = (self.lh, neigh)
                if i == 3:
                    new_state.rh = Limb(LimbName.RIGHT_HAND, 8, 2, neigh)
                    action = (self.rh, neigh)
                succs.append((new_state, action))
        #print(succs)
        return succs

    def getNeighbors(self, limb):
        neighbors = []
        for hold in self.route.holds:
            if not hold == limb.hold:
                if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                    #print("Checking leg neighbors")
                    #print(f"Height diff: {abs(hold.y - limb.hold.y)}")
                    #print(f"Leg length: {self.inches_to_pixels(self.person.leg_length)}")
                    # low_arm is max because pixels go from top to bottom
                    low_arm = max([self.lh.hold.y, self.rh.hold.y])
                    if (0 < limb.hold.y - hold.y < self.inches_to_pixels(self.person.leg_length) and 
                        abs(hold.x - limb.hold.x) < self.inches_to_pixels(self.person.leg_length) and
                        hold.y > low_arm): 
                        neighbors.append(hold)
                elif limb.name in [LimbName.LEFT_HAND, LimbName.RIGHT_HAND]:
                    #print("Checking arm neighbors")
                    #print("HAND TIME 1")
                    upper_leg, lower_leg = sorted([self.lf.hold.y, self.rf.hold.y])
                    if (abs(hold.x - limb.hold.x) < self.inches_to_pixels(self.person.wingspan) and
                        lower_leg - hold.y < self.inches_to_pixels(self.person.height * 0.8) and hold.y < upper_leg and 0 < limb.hold.y - hold.y):
                        #print(hold.y - upper_leg)
                        #print("HAND TIME 2")
                        neighbors.append(hold)
        return neighbors
        
def moveDifficulty(state: State, limb: Limb, next_hold: Hold):
    distance = math.sqrt(((limb.hold.x - next_hold.x) ** 2) + ((limb.hold.y - next_hold.y) ** 2))
    distance_diff = distance

    new_state = State(copy(state.lf), copy(state.rf), copy(state.lh), copy(state.rh), state.person, state.route)
    new_state_limbs = [new_state.lh, new_state.rh, new_state.lf, new_state.rf]
    for new_state_limb in new_state_limbs:
        if new_state_limb.name == limb.name:
            new_state_limb.hold = None
    state_without_limb_difficulty = 0.3 * stateDifficulty(new_state)
    distance_diff *= 0.1
    move_diff = distance_diff + state_without_limb_difficulty
    #print( distance_diff)
    return move_diff

def stateDifficulty(state: State):
    if state.lh.hold != None and state.rh.hold != None:
        average_hands_x = (state.lh.hold.x + state.rh.hold.x) / 2
        average_hands_y = (state.lh.hold.y + state.rh.hold.y) / 2
    else:
        if state.rh.hold == None:
            average_hands_x = state.lh.hold.x  
            average_hands_y = state.lh.hold.y
        else:
            average_hands_y = state.rh.hold.y
            average_hands_x = state.rh.hold.x
    if state.lf.hold != None and state.rf.hold != None:
        average_legs_x = (state.lf.hold.x + state.rf.hold.x) / 2
        average_legs_y = (state.lf.hold.y + state.rf.hold.y) / 2 
    else:
        if state.rf.hold == None:
            average_legs_x = state.lf.hold.x
            average_legs_y = state.lf.hold.y
        else:
            average_legs_y = state.rf.hold.y
            average_legs_x = state.rf.hold.x

    hands_difference_x = abs(state.rh.hold.x - state.lh.hold.x)if state.lh.hold != None and state.rh.hold != None else 0
    hands_difference_y = abs(state.rh.hold.y - state.lh.hold.y) if state.lh.hold != None and state.rh.hold != None else 0

    legs_difference_x = abs(state.rf.hold.x - state.lf.hold.x) if state.lf.hold != None and state.rf.hold != None else 0
    legs_difference_y = abs(state.lf.hold.y - state.rf.hold.y) if state.lf.hold != None and state.rf.hold != None else 0

    hands_difference_raw_x = state.rh.hold.x - state.lh.hold.x if state.lh.hold != None and state.rh.hold != None else 0
    legs_difference_raw_x = state.rf.hold.x - state.lf.hold.x if state.lf.hold != None and state.rf.hold != None else 0

    leg_match_diff = 0
    if state.lf.hold != None and state.rf.hold != None and state.lf.hold.x - state.rf.hold.x == 0:
        leg_match_diff = 50

    cross_diff = 0
    if hands_difference_raw_x < 0:
        cross_diff += 2 * abs(hands_difference_raw_x / 88)
    if legs_difference_raw_x < 0:
        cross_diff += 100
    if hands_difference_raw_x < 0 and legs_difference_raw_x< 0:
        cross_diff *= 3

    diff = 0
    center_diff = abs(average_hands_x - average_legs_x) ** 2

    target_distance = state.inches_to_pixels(state.person.height * 0.8)
    distance_diff = target_distance - abs(average_legs_y - average_hands_y)
    raw_difficulty_score = (distance_diff ** 2)

    limb_strength_diff = 0
    angle_diff = 0
    for limb in [state.lh, state.rh, state.lf, state.rf]:
        if limb.hold != None:
            limb_strength_diff += limb.hold.diff / limb.strength
            if limb.name == LimbName.LEFT_HAND:
                if 315 >= limb.hold.angle >= 270:
                    angle_diff += 2
                elif 90 >= limb.hold.angle or limb.hold.angle > 315:
                    angle_diff += 1
                elif 180 >= limb.hold.angle > 90:
                    angle_diff += 2.5
                else:
                    angle_diff += 3
            if limb.name == LimbName.RIGHT_HAND:
                if 90 >= limb.hold.angle >= 45:
                    angle_diff += 2
                elif 45 >= limb.hold.angle or limb.hold.angle > 270:
                    angle_diff += 1
                elif 270 >= limb.hold.angle > 180:
                    angle_diff += 2.5
                else:
                    angle_diff += 3
            if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                if 90 <= limb.hold.angle <= 270:
                    angle_diff += 2
        else:
            limb_strength_diff += 6
    separation_diff = 0
    separation_diff += 0.1 * hands_difference_y
    separation_diff += 0.1 * legs_difference_y



    if hands_difference_x > state.inches_to_pixels(0.8 * state.person.wingspan):
        separation_diff += 0.5 * hands_difference_x
    separation_diff += 0.5 * legs_difference_y
    if legs_difference_x > state.inches_to_pixels(0.6 * state.person.wingspan):
        separation_diff += 0.5 * legs_difference_x
    center_diff *= .04
    raw_difficulty_score *= 0.005
    angle_diff *= 1
    limb_strength_diff *= 30
    separation_diff *= 0.5
    cross_diff *= 1
    leg_match_diff *=12
    diff += center_diff + raw_difficulty_score + angle_diff + limb_strength_diff + separation_diff + leg_match_diff + cross_diff
    #print(f"center = {center_diff}, scaled = {raw_difficulty_score}, angle = {angle_diff}, strength = {limb_strength_diff}, cross = {cross_diff}")
    return diff

In [94]:
import numpy as np
from copy import copy
class aStar:
    WALL_HEIGHT = 180 #image.height -> divide that by height of wall in inches to get conversion
    def __init__(self, state):
        self.state = state
        self.reach = state.person.reach

    
                    
    def get_distance(self, hold, state):
        return np.sqrt((state[0] - hold[0]) ** 2 + (state[1] - hold[1]) ** 2)

    def getCostValue(self, costs):
        return sum([cost for cost in costs]) + 250 * len(costs)
        
    def uniformCostSearch(self):
        explored = []
        frontier = PriorityQueue()
        num = 0
        frontier.push(self.state, 0)
        while not frontier.isEmpty():
            cur_state = frontier.pop()
            #print(cur_state.moves)
            # print(f"Looking at state {cur_state}")
            explored.append(cur_state)
            if cur_state.lh.hold == self.state.route.finish_hold and cur_state.rh.hold == self.state.route.finish_hold:
                print("TAAAAAAAAAAAAAAKE")
                print(cur_state.costs)
                print(self.getCostValue(cur_state.costs))
                return cur_state.moves
            for next_state, action in cur_state.getStateSuccessors():
                if next_state not in explored: 
                    num += 1
                    if num % 500 == 0:
                        print(f"Checking state #{num} with move length {len(cur_state.moves)}")
                    #print("not in explored")
                    next_state.costs = copy(cur_state.costs)
                    next_state.costs.append(stateDifficulty(next_state) + moveDifficulty(cur_state, action[0], action[1]))
                    next_state.moves = copy(cur_state.moves)
                    next_state.moves.append(action)
                    #print(next_state.moves)
                    #print(frontier.heap)
                    if next_state in [tup[2] for tup in frontier.heap]:
                        frontier.update(next_state, self.getCostValue(next_state.costs))
                        #print(f"Updated {next_state} with {self.getCostValue(next_state.costs)}")
                    else:
                        frontier.push(next_state, self.getCostValue(next_state.costs))
                        #print(f"pushed {next_state} with {self.getCostValue(next_state.costs)}")
        return []


In [132]:
import get_holds
from get_holds import getHoldsArray
import importlib
importlib.reload(get_holds)
holds = getHoldsArray('../images/traverse.jpg', [270, 5, 21], 50)

holds.sort()
print(holds)
print(len(holds))


image 1/1 c:\Users\liaml\AI-Climbing\src\..\images\traverse.jpg: 1280x1120 130 holds, 405.2ms
Speed: 27.5ms preprocess, 405.2ms inference, 3.0ms postprocess per image at shape (1, 3, 1280, 1120)

0: 64x64 six 0.69, four 0.16, five 0.09, three 0.03, one 0.02, 6.0ms
Speed: 0.0ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 64, 64)
running

0: 64x64 four 0.43, seven 0.18, five 0.14, three 0.11, two 0.06, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 64, 64)
running

0: 64x64 four 0.54, five 0.25, one 0.07, eight 0.06, seven 0.02, 4.0ms
Speed: 0.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 64, 64)
running

0: 64x64 five 0.31, ten 0.17, four 0.16, seven 0.11, two 0.09, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 64, 64)
running

0: 64x64 eight 0.75, seven 0.11, four 0.10, three 0.02, two 0.00, 4.5ms
Speed: 1.0ms preprocess, 4.5ms inferenc

In [73]:
def moveToText(action, route):
    output = "- Move your "
    if action[0].name == LimbName.LEFT_LEG:
        output += "left leg "
    elif action[0].name == LimbName.LEFT_HAND:
        output += "left hand "
    elif action[0].name == LimbName.RIGHT_LEG:
        output += "right leg "
    elif action[0].name == LimbName.RIGHT_HAND:
        output += "right hand "
    output += "to hold number "
    output += str(route.holds.index(action[1]))
    output += " from the top"
    return output

In [139]:
route = Route(holds = holds, start1 = holds[10], start2 = holds[10], finish = holds[0])
lh = Limb(LimbName.LEFT_HAND, 2.5, 8, route.start_hold1)
rh = Limb(LimbName.RIGHT_HAND, 2.5, 8, route.start_hold2)
lf = Limb(LimbName.LEFT_LEG, 8, 5, route.holds[-1])
rf = Limb(LimbName.RIGHT_LEG, 8, 5, route.holds[-1])
ondra = Person(70)
liam = Person(68)
rachel = Person(66)
anna = Person(63)
short = Person(59)
luisa = Person(64)
tall = Person(78)
giga = Person(75)
state1 = State(lf, rf, lh, rh, ondra, route)
a_star1 = aStar(state=state1)
results1 = a_star1.uniformCostSearch()

for action in results1:
    print(moveToText(action, route))

# for action in aStar(state=State(lf, rf, lh, rh, tall, route)).uniformCostSearch():
#     print(moveToText(action, route))

Checking state #500 with move length 4
Checking state #1000 with move length 3
Checking state #1500 with move length 3
Checking state #2000 with move length 5
Checking state #2500 with move length 4
Checking state #3000 with move length 5
Checking state #3500 with move length 5
Checking state #4000 with move length 6
Checking state #4500 with move length 5
Checking state #5000 with move length 6
Checking state #5500 with move length 4
Checking state #6000 with move length 5
Checking state #6500 with move length 7
Checking state #7000 with move length 9
Checking state #7500 with move length 4
Checking state #8000 with move length 8
TAAAAAAAAAAAAAAKE
[1504.2019672921263, 1248.7379168579855, 2472.870295878658, 1128.2386396765824, 1690.5151952737806, 4322.230269706546, 3293.007093988982, 969.3638197217903, 1130.4186095866728, 1868.3064912484012, 1454.1433088972417, 865.115514596426, 1010.7079842551539, 2608.7279388022334, 2329.998705739506, 2120.1840445215607]
34016.76779604364
- Move your

In [133]:
print(len(holds))
for i in range(len(holds)):
    print(f"{i}: {holds[i].x}, {holds[i].y}, {holds[i].diff}")



21
0: 2433.7822265625, 853.4515380859375, 6.017306327819824
1: 2318.96337890625, 1101.709228515625, 4.843441486358643
2: 2092.406494140625, 1113.8349609375, 2.2649447917938232
3: 2302.6796875, 1416.130859375, 4.408835411071777
4: 2297.79443359375, 1627.1378173828125, 2.405978202819824
5: 2186.1572265625, 1926.5257568359375, 3.651273727416992
6: 2277.638916015625, 2034.423828125, 5.430213928222656
7: 2628.416015625, 2120.174560546875, 2.3968923091888428
8: 1787.193603515625, 2147.49072265625, 10.0
9: 634.6383056640625, 2148.4912109375, 10.0
10: 2614.080078125, 2319.30859375, 3.2914018630981445
11: 2193.6005859375, 2331.91845703125, 8.640486717224121
12: 2500.1123046875, 2609.57666015625, 8.746700286865234
13: 2100.5009765625, 2768.98486328125, 10.0
14: 2100.64111328125, 2770.2978515625, 10.0
15: 563.5315551757812, 2859.7412109375, 5.6248321533203125
16: 2594.3896484375, 2879.353271484375, 9.422250747680664
17: 1910.308349609375, 2925.443603515625, 10.0
18: 2296.93994140625, 2948.5920410

In [14]:
import os
print(os.path.join('runs', 'yo'))

runs\yo


In [15]:
import sys
print(sys.version)

3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:18:13) [MSC v.1916 64 bit (AMD64)]
